# Data Wrangling with OpenStreetMaps and SQL

OpenStreetMap is a volunteered geographic information project that runs on open source collaboration to provide free and editable maps to everyone on this planet (assuming they have internet access). For more information, see <a href="https://en.wikipedia.org/wiki/OpenStreetMap">OpenStreetMap: Wikipedia Article</a>.

In OpenStreetMap, data is stored in an xml format (for more information, see <a href="http://wiki.openstreetmap.org/wiki/OSM_XML">OpenStreetMap xml-Wiki</a>)

In many cases of user generated content, the probability of errors or inconsistencies in data quality is given. The goal of this project is to apply data wrangling techniques:

<ul>
  <li>Audit map data of the city of Phoenix Arizona</li>
  <li>Fix problems encountered and enhance data quality</li>
  <li>Import data into an sqlite database to perform some exploratory queries</li>
</ul>


### Chosen Map Area

The chosen map area is the <a href="http://www.openstreetmap.org/export#map=10/33.6049/-112.1248">City of Phoenix AZ (USA)</a> - latitude 34.1141 and 33.0938, longitude 113.2031 and -111.0471. I have chosen Phoenix for this project because as a teenager I have lived there and know the city quite well.

The data for this project was retrieved from <a href="https://mapzen.com/data/metro-extracts/metro/phoenix_arizona/">Mapzen - Metro Extracts</a>. The overall filesize of the unzipped xml file is ~605mb.

### Auditing the Data

After downloading the xml file the data wrangling process was initiated by looking at

<ul>
  <li>Street Names</li>
  <li>Postal Codes</li>
  <li>Phone Numbers</li>
</ul>

The goal was to assess the data quality for each of the three and to enhance the data quality by implementing changes for
a more standardized way of information display.

### Problem with the Data - Street Name Abbreviations

For Street names a non consistent usage of street name abbreviations was expected.

In regard to data cleaning the plan for action was as followed:

<ul>
  <li>Use regex to match common street name endings (like Road, Drive, Street..)</li>
  <li>Create a list of street name endings that do not need to be cleaned (like Road, Drive, Street..)
  <li>Parse through the XML looking for tag elements with k="addr:street" attributes</li>
  <li>Build key dictionary with matches of the street types (with regex) and a set of street names with identical keys</li>
  <li>Create a mapping for street name cleaning
  <li>Build a function that will clean street names 
</ul>




In [13]:
for street_type, ways in street_types.iteritems(): 
        for name in ways:
            if "Suite"  in name:
                name = name.split(", Suite")[0].strip()
            if "#" in name:
                name = name.split(" #")[0].strip()
            if "," in name:
                name = name.split(", ")[0].strip()
            if "Suite" in name:
                name = name.split(" Suite")[0].strip()
            if "Building" in name:
                name = name.split(" Building")[0].strip()
            if "Ste" in name:
                name = name.split(" Ste")[0].strip()
            if "St" in name:
                name = name.split(" St")[0].strip()
            name_improv_first = update_name(name, mapping, street_type_re)
            name_improv_sec = update_name(name_improv_first, mapping2, street_type_pre)
            
            print name, "=>", name_improv_first, "=>", name_improv_sec

East Pinnacle Peak Road => East Pinnacle Peak Road => East Pinnacle Peak Road
N. Hayden Road => N. Hayden Road => North Hayden Road
West Happy Valley Rd => West Happy Valley Road => West Happy Valley Road
E Warner Rd => E Warner Road => East Warner Road
East Williamsfield Rd => East Williamsfield Road => East Williamsfield Road
E. Elliot Rd => E. Elliot Road => East Elliot Road
N Hayden Rd => N Hayden Road => North Hayden Road
S Watson Rd => S Watson Road => South Watson Road
W Camelback Rd => W Camelback Road => West Camelback Road
N Scottsdale Rd => N Scottsdale Road => North Scottsdale Road
E Baseline Rd => E Baseline Road => East Baseline Road
E. Marilyn Rd. => E. Marilyn Road => East Marilyn Road
N 32nd => N 32nd => North 32nd
East Avenue of the Fountains => East Avenue of the Fountains => East Avenue of the Fountains
E. Brown RD => E. Brown Road => East Brown Road
South Sirrine => South Sirrine => South Sirrine
East Highway 287 => East Highway 287 => East Highway 287
17054 W Post

<ul>
  <li>N Hayden Rd => N Hayden Road => North Hayden Road</li>
  <li>S Watson Rd => S Watson Road => South Watson Road</li>
  <li>E. Elliot Rd => E. Elliot Road => East Elliot Road</li>
  <li>...</li>
</ul>

This updated all substrings, such that: “S Watson Rd” becomes: “South Watson Road”. It is important to note that this does not mean, all street adresses for the city of Phoenix have been cleaned now. We used a certain lens via regex and expected street type endings to retrieve a set of street types to change accordingly. There is still a blindspot depending on the lens chosen.

### Problem with the Data - Postal Codes

For Postal Codes we used a similar approach as in the Street Name Cleaning. The Goal was to only have 5 Digits Postal Codes 
without any additional info.

The changes were similar to the following:

<ul>
  <li>AZ 85023 => 85023</li>
  <li>85007, AZ => 85007</li>
  <li>82158, 123 => 82158</li>
  <li>...</li>
</ul>

### Problem with the Data - Phone Numbers

In the same way, Phone Numbers were cleaned so that the info was displayed as "XXX XXX XXXX". All in all, the phone numbers were quite messy. 

The changes were similar to the following:

<ul>
  <li>623-546-1640 => 623 546 1640</li>
  <li>+1 4806714584 => 480 671 4584</li>
  <li>(602) 734 5106 => 602 734 5106</li>
  <li>...</li>
</ul>

In the next step, we create 5 csv files for nodes_path, node_tags, ways, ways_nodes and ways_tags to later insert into sql. 

### Overview of the Data

#### File Sizes

In [85]:
import os
print 'The file downloaded for the map data of the city of Phoenix is {} MB'.format(os.path.getsize("Phoenix_Arizona.osm")/1.0e6)
print 'The csv file for nodes is {} MB'.format(os.path.getsize("nodes.csv")/1.0e6)
print 'The csv file for nodes_tags is {} MB'.format(os.path.getsize("nodes_tags.csv")/1.0e6)
print 'The csv file for ways is {} MB'.format(os.path.getsize("ways.csv")/1.0e6)
print 'The csv file for ways_nodes is {} MB'.format(os.path.getsize("ways_nodes.csv")/1.0e6)
print 'The csv file for ways_tags is {} MB'.format(os.path.getsize("ways_tags.csv")/1.0e6)
print 'The db file for OpenstreetMap_Phx  is {} MB'.format(os.path.getsize("OpenStreetMap_Phx.db")/1.0e6)

The file downloaded for the map data of the city of Phoenix is 619.829206 MB
The csv file for nodes is 225.59075 MB
The csv file for nodes_tags is 10.902282 MB
The csv file for ways is 21.286642 MB
The csv file for ways_nodes is 79.504337 MB
The csv file for ways_tags is 66.319958 MB
The db file for OpenstreetMap_Phx  is 368.549888 MB


#### Number of Nodes

In [90]:
sqlite> SELECT COUNT(*) FROM nodes;

Number of nodes is:[(2643915,)]


The number of nodes is 2643915.

#### Number of Ways

In [91]:
sqlite>   SELECT COUNT(*) FROM ways;

Number of ways is:[(349223,)]


#### Number of Unique Users

In [89]:
sqlite> SELECT COUNT(DISTINCT(e.uid))          
        FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;

Number of unique users is:[(1107,)]


The number of unique users is 1107.

#### Top 10 contributing Users

In [92]:
sqlite> SELECT e.user, COUNT(*) as num
        FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
        GROUP BY e.user
        ORDER BY num DESC
        LIMIT 10;

Number of unique users are:
[(u'Dr Kludge', 1073635),
 (u'TheDutchMan13', 320948),
 (u'$user', 200492),
 (u'tomthepom', 114751),
 (u'adenium', 100080),
 (u'CartoCrazy', 71274),
 (u'woodpeck_fixbot', 63224),
 (u'kghazi', 61994),
 (u'namannik', 45075),
 (u'Adam Martin', 40184)]


#### Number of Unique Users that appear only 5 times or less

In [95]:
sqlite> SELECT COUNT(*) 
        FROM
        (SELECT e.user, COUNT(*) as num
         FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) as e
         GROUP BY e.user
         HAVING num<=5)  u;

Number of unique users only appearing once are:
[(392,)]


The number of 392 is quite low compared to a total of 1107 unique users (~34 Percent). This seems to indicate that Phoenix has a quite active user base with a majority of unique users making larger contributions.

#### Top 5 religious amenities

In [99]:
sqlite> SELECT nodes_tags.value, COUNT(*) as num
        FROM nodes_tags 
        JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') as i
        ON nodes_tags.id=i.id
        WHERE nodes_tags.key='religion'
        GROUP BY nodes_tags.value
        ORDER BY num DESC
        LIMIT 5;

[(u'christian', 560),
 (u'jewish', 4),
 (u'bahai', 2),
 (u'buddhist', 1),
 (u'muslim', 1)]


At least by looking at the OpenStreetMap Data, Phoenix does not have many non-Christian amenities. Only about 1,4 Percent of the
TOP 5 religious amenities are non Christian.

#### TOP 10 Cuisines

In [100]:
sqlite> SELECT nodes_tags.value, COUNT(*) as num
        FROM nodes_tags 
        JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') as i
        ON nodes_tags.id=i.id
        WHERE nodes_tags.key='cuisine'
        GROUP BY nodes_tags.value
        ORDER BY num DESC
        LIMIT 10;

[(u'pizza', 47),
 (u'american', 42),
 (u'mexican', 42),
 (u'sandwich', 27),
 (u'italian', 25),
 (u'burger', 21),
 (u'chinese', 15),
 (u'sushi', 11),
 (u'asian', 8),
 (u'greek', 7)]


#### TOP 10 Leisure Ameneties

In [101]:
sqlite> SELECT nodes_tags.value, count(*) as num
        FROM nodes_tags 
        WHERE nodes_tags.key=='leisure'
        GROUP BY nodes_tags.value
        ORDER BY num DESC

[(u'swimming_pool', 1506),
 (u'playground', 387),
 (u'park', 322),
 (u'picnic_table', 129),
 (u'sports_centre', 37),
 (u'pitch', 27),
 (u'slipway', 21),
 (u'water_park', 8),
 (u'fitness_centre', 7),
 (u'marina', 5)]


### Other Ideas about the Dataset

Exploring the dataset I am curious about additional information that is currently not included in the data - for example - What are average housing prices in different areas of the city of Phoenix? Which playground are of high quality and which might need additional renovation and are therefore less recommended going to with the kids?

In general, this could be thought as a recommendation system with ratings on restaurants and other amenities. The benefit of such a recommendation system is straightforward - having an idea of places worthy to go / or even live. Implementing a system like this would have one major downside - the recommendations would be highly subjective in regard to the user adding the information and could also be outdated quickly if not updated regurlarly. 

A general feature that would be very valuable for the OpenStreetMap Community to have is visualized information, which parts of the city have been updated recently and which parts could need additional updates? One way to display this could be a heatmap showing how a region has been updated (frequency / time). The benefit for the community would be to quickly see which regions to focus on for maximum benefit of overall map quality.

### References

Udacity Discussion Forum (extensively)<br>
<a href="https://docs.python.org/2/library/os.path.html">Python Docs - os.path</a><br>
<a href="http://www.w3schools.com/sql/">w3schools - SQL</a><br>
<a href="https://www.stackoverflow.com">Stackoverflow Forum</a><br>
<a href="https://regex101.com">Regex101</a><br>
<a href="https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md">Github - Sample Project</a>
